# SETUP

## is GPU on?

In [ ]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

## boilerplate

In [1]:
from tensorflow.keras.callbacks import Callback, EarlyStopping
from numpy import arange

MODEL_NAME = 'resnet50'
IMG_SIZE = (224, 224)
INPUT_SHAPE=(224, 224, 3)
CLASSES = 2
TRIAL = list(arange(0,3))
FT_BLOCK = 8 # FROM feature extractor TO fine tuning scratch
BATCH_SIZE = 32 # these data points will be passed as a batch at one time to the network
PR_RATIO = list(arange(1,10,1))
TRIAL = list(arange(0,10,1))
MODEL_DIR = '../../models/PR'

## UDC/Fs

In [2]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import layers
from math import floor
from timeit import default_timer as timer
from json import dump
from tensorflow.keras.models import Model

def model_constructor(FT_BLOCK):
    base_model = ResNet50(
        weights='imagenet', 
        include_top=False,
        input_shape=INPUT_SHAPE)
    
    # construct the top layer containing 
    x = base_model.output
    x = layers.GlobalAveragePooling2D(name='avg_pool')(x) # add a global spatial average pooling layer
    x = layers.Dense(CLASSES, activation='sigmoid',  name='predictions')(x) # and add the output layer for binary class

    # model is ready to be trained
    model = Model(inputs=base_model.input, outputs=x)

    # freeze the layers before the `FROZEN_END` layer 
    total_layers = len(base_model.layers)
    ft_layers = floor(total_layers / 10)
    FROZEN_END = total_layers - ft_layers * FT_BLOCK

    base_model.trainable = True
    for layer in base_model.layers[:FROZEN_END]: 
        layer.trainable = False
    
    return model

class TimeCallback(Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

def time_converter(sec):
    hours, rem = divmod(sec, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

def save_history(history, tag):
    file_path = f'{HISTORY_DIR}/{tag}.json'
    with open(file_path, 'w') as f:
        dump(history.history, f)
        
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow import data
def data_preparation(BATCH_SIZE):
    train_dir = '../../data/binaryclass_clean/train/'
    test_dir = '../../data/binaryclass_clean/test/'

    train_ds = image_dataset_from_directory(
        directory=train_dir,
        label_mode='categorical',
        batch_size=BATCH_SIZE,
        image_size=IMG_SIZE,
        seed=0,
        validation_split=0.1,
        subset='training')

    val_ds = image_dataset_from_directory(
        directory=train_dir,
        label_mode='categorical',
        batch_size=BATCH_SIZE,
        image_size=IMG_SIZE,
        seed=0,
        validation_split=0.1,
        subset='validation')

    test_ds = image_dataset_from_directory(
        directory=test_dir,
        label_mode='categorical',
        batch_size=1,
        image_size=IMG_SIZE)

    AUTOTUNE = data.AUTOTUNE
    train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
    
    return train_ds, val_ds, test_ds

import tempfile
from os.path import getsize

def get_gzipped_model_size(file):
  # Returns size of gzipped model, in Mbytes.
  import os
  import zipfile

  zipped_file = f'{MODEL_DIR}/{tag}.zip'
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)
    zipped_size = getsize(zipped_file) / float(2**20)
#     remove(zipped_file)

  return zipped_size

## load data

In [3]:
train_ds, val_ds, test_ds = data_preparation(BATCH_SIZE)

Found 9980 files belonging to 2 classes.
Using 8982 files for training.
Found 9980 files belonging to 2 classes.
Using 998 files for validation.
Found 1000 files belonging to 2 classes.


# GRID SEARCH

In [4]:
# !pip install --user --upgrade tensorflow-model-optimization

In [ ]:
import tensorflow_model_optimization as tfmot
from tensorflow.keras.models import save_model

for i in TRIAL:
    print(" - - - - - TRIAL:", i, " - - - - - ")
    loss, accuracy = [None]*len(PR_RATIO), [None]*len(PR_RATIO)

    for pr in PR_RATIO:
        tag = f'{MODEL_NAME}_BS{BATCH_SIZE}_FT{FT_BLOCK*10}_PR{pr*10}_{i}'
        print(tag)

        logdir = f'../../logs/PR/{tag}.json'
        callbacks = [
            tfmot.sparsity.keras.UpdatePruningStep(),
            tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
        ]
        pruning_params = {
            'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(
                target_sparsity=pr/10, begin_step=0, end_step=-1, frequency=1)}

        model = model_constructor(FT_BLOCK)
        prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
        model_for_pruning = prune_low_magnitude(model, **pruning_params)
        # model_for_pruning.summary()

        model_for_pruning.compile(loss='binary_crossentropy', metrics='accuracy', optimizer = 'adam')
        history = model_for_pruning.fit(train_ds, epochs=100, verbose=0, validation_data=val_ds, callbacks=callbacks)

        loss[pr-1], accuracy[pr-1] = model_for_pruning.evaluate(test_ds)

    # PRINT THE RESULT
    print(f' * * * * * {tag[:-2]}10-90_LOSS * * * * * ')
    for item in loss:
        print(item)
    print(f' * * * * * {tag[:-2]}10-90_ACCURACY * * * * * ')
    for item in accuracy:
        print(item)

 - - - - - TRIAL: 0  - - - - - 
resnet50_BS32_FT80_PR10_0
1000/1000 [==============================] - 25s 25ms/step - loss: 1.0889 - accuracy: 0.8820
resnet50_BS32_FT80_PR20_0
1000/1000 [==============================] - 23s 23ms/step - loss: 0.7058 - accuracy: 0.8920
resnet50_BS32_FT80_PR30_0
1000/1000 [==============================] - 24s 23ms/step - loss: 0.8996 - accuracy: 0.8920
resnet50_BS32_FT80_PR40_0
1000/1000 [==============================] - 24s 24ms/step - loss: 0.9194 - accuracy: 0.8900
resnet50_BS32_FT80_PR50_0
1000/1000 [==============================] - 24s 24ms/step - loss: 0.9346 - accuracy: 0.9050
resnet50_BS32_FT80_PR60_0
1000/1000 [==============================] - 24s 24ms/step - loss: 0.6684 - accuracy: 0.9090
resnet50_BS32_FT80_PR70_0
1000/1000 [==============================] - 22s 22ms/step - loss: 0.9152 - accuracy: 0.9080
resnet50_BS32_FT80_PR80_0
1000/1000 [==============================] - 22s 22ms/step - loss: 0.7082 - accuracy: 0.9040
resnet50_BS32_FT

1000/1000 [==============================] - 17s 17ms/step - loss: 0.9972 - accuracy: 0.8950
resnet50_BS32_FT80_PR80_2
1000/1000 [==============================] - 17s 17ms/step - loss: 0.6153 - accuracy: 0.9140
resnet50_BS32_FT80_PR90_2
1000/1000 [==============================] - 17s 17ms/step - loss: 0.8599 - accuracy: 0.8830
 * * * * * resnet50_BS32_FT80_PR9010-90_LOSS * * * * * 
0.981147289276123
0.7743844985961914
0.8219264149665833
0.9516041278839111
0.7973278760910034
0.6470641493797302
0.9972418546676636
0.6152729988098145
0.8599050641059875
 * * * * * resnet50_BS32_FT80_PR9010-90_ACCURACY * * * * * 
0.8930000066757202
0.9089999794960022
0.9070000052452087
0.9089999794960022
0.8960000276565552
0.921999990940094
0.8949999809265137
0.9139999747276306
0.8830000162124634
 - - - - - TRIAL: 3  - - - - - 
resnet50_BS32_FT80_PR10_3
1000/1000 [==============================] - 17s 17ms/step - loss: 0.8485 - accuracy: 0.8750
resnet50_BS32_FT80_PR20_3
1000/1000 [=========================

In [4]:
TRIAL = list(arange(7,10,1))

In [6]:
import tensorflow_model_optimization as tfmot
from tensorflow.keras.models import save_model

for i in TRIAL:
    print(" - - - - - TRIAL:", i, " - - - - - ")
    loss, accuracy = [None]*len(PR_RATIO), [None]*len(PR_RATIO)

    for pr in PR_RATIO:
        tag = f'{MODEL_NAME}_BS{BATCH_SIZE}_FT{FT_BLOCK*10}_PR{pr*10}_{i}'
        print(tag)

        logdir = f'../../logs/PR/{tag}.json'
        callbacks = [
            tfmot.sparsity.keras.UpdatePruningStep(),
            tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
        ]
        pruning_params = {
            'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(
                target_sparsity=pr/10, begin_step=0, end_step=-1, frequency=1)}

        model = model_constructor(FT_BLOCK)
        prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
        model_for_pruning = prune_low_magnitude(model, **pruning_params)
        # model_for_pruning.summary()

        model_for_pruning.compile(loss='binary_crossentropy', metrics='accuracy', optimizer = 'adam')
        history = model_for_pruning.fit(train_ds, epochs=100, verbose=0, validation_data=val_ds, callbacks=callbacks)

        loss[pr-1], accuracy[pr-1] = model_for_pruning.evaluate(test_ds)

    # PRINT THE RESULT
    print(f' * * * * * {tag[:-4]}10-90_LOSS * * * * * ')
    for item in loss:
        print(item)
    print(f' * * * * * {tag[:-4]}10-90_ACCURACY * * * * * ')
    for item in accuracy:
        print(item)

 - - - - - TRIAL: 7  - - - - - 
resnet50_BS32_FT80_PR10_7
1000/1000 [==============================] - 19s 19ms/step - loss: 0.7221 - accuracy: 0.9000
resnet50_BS32_FT80_PR20_7
1000/1000 [==============================] - 20s 20ms/step - loss: 0.8005 - accuracy: 0.9110
resnet50_BS32_FT80_PR30_7
1000/1000 [==============================] - 20s 20ms/step - loss: 0.9701 - accuracy: 0.8910
resnet50_BS32_FT80_PR40_7
1000/1000 [==============================] - 19s 19ms/step - loss: 0.7395 - accuracy: 0.9060
resnet50_BS32_FT80_PR50_7
1000/1000 [==============================] - 20s 20ms/step - loss: 1.2751 - accuracy: 0.8660
resnet50_BS32_FT80_PR60_7
1000/1000 [==============================] - 20s 19ms/step - loss: 0.6877 - accuracy: 0.9180
resnet50_BS32_FT80_PR70_7
1000/1000 [==============================] - 20s 20ms/step - loss: 0.8255 - accuracy: 0.9150
resnet50_BS32_FT80_PR80_7
1000/1000 [==============================] - 20s 20ms/step - loss: 0.8874 - accuracy: 0.8790
resnet50_BS32_FT

1000/1000 [==============================] - 20s 20ms/step - loss: 0.8832 - accuracy: 0.9080
resnet50_BS32_FT80_PR80_9
1000/1000 [==============================] - 19s 19ms/step - loss: 0.6421 - accuracy: 0.9050
resnet50_BS32_FT80_PR90_9
1000/1000 [==============================] - 20s 20ms/step - loss: 0.6735 - accuracy: 0.9060
 * * * * * resnet50_BS32_FT80_PR10-90_LOSS * * * * * 
0.9065030813217163
1.5966176986694336
0.9469444751739502
1.0058196783065796
0.7363739013671875
1.4485546350479126
0.8832193613052368
0.6420564651489258
0.6734665036201477
 * * * * * resnet50_BS32_FT80_PR10-90_ACCURACY * * * * * 
0.8849999904632568
0.8659999966621399
0.8880000114440918
0.8870000243186951
0.9129999876022339
0.8629999756813049
0.9079999923706055
0.9049999713897705
0.906000018119812


# MODEL WITHOUT PRUNE

In [6]:
model = model_constructor(FT_BLOCK)
model.compile(loss='binary_crossentropy', metrics='accuracy', optimizer='adam')
model.fit(train_ds, epochs=100, verbose=1, validation_data=val_ds)
model.evaluate(test_ds)

Epoch 1/100
281/281 [==============================] - 43s 127ms/step - loss: 0.3828 - accuracy: 0.8479 - val_loss: 0.4035 - val_accuracy: 0.8697
Epoch 2/100
281/281 [==============================] - 36s 124ms/step - loss: 0.2270 - accuracy: 0.9067 - val_loss: 0.2806 - val_accuracy: 0.9118
Epoch 3/100
281/281 [==============================] - 35s 123ms/step - loss: 0.1862 - accuracy: 0.9247 - val_loss: 0.2459 - val_accuracy: 0.9118
Epoch 4/100
281/281 [==============================] - 35s 122ms/step - loss: 0.1597 - accuracy: 0.9344 - val_loss: 0.2371 - val_accuracy: 0.9248
Epoch 5/100
281/281 [==============================] - 35s 123ms/step - loss: 0.1333 - accuracy: 0.9436 - val_loss: 0.4227 - val_accuracy: 0.8918
Epoch 6/100
281/281 [==============================] - 35s 122ms/step - loss: 0.1327 - accuracy: 0.9436 - val_loss: 0.2512 - val_accuracy: 0.9128
Epoch 7/100
281/281 [==============================] - 35s 123ms/step - loss: 0.0963 - accuracy: 0.9561 - val_loss: 0.2278 -

Epoch 57/100
281/281 [==============================] - 36s 125ms/step - loss: 0.0069 - accuracy: 0.9969 - val_loss: 0.3804 - val_accuracy: 0.9329
Epoch 58/100
281/281 [==============================] - 35s 122ms/step - loss: 0.0081 - accuracy: 0.9967 - val_loss: 0.4597 - val_accuracy: 0.9349
Epoch 59/100
281/281 [==============================] - 35s 123ms/step - loss: 0.0248 - accuracy: 0.9906 - val_loss: 0.5198 - val_accuracy: 0.9168
Epoch 60/100
281/281 [==============================] - 37s 130ms/step - loss: 0.0180 - accuracy: 0.9934 - val_loss: 0.4449 - val_accuracy: 0.9228
Epoch 61/100
281/281 [==============================] - 35s 122ms/step - loss: 0.0129 - accuracy: 0.9947 - val_loss: 0.4129 - val_accuracy: 0.9319
Epoch 62/100
281/281 [==============================] - 35s 123ms/step - loss: 0.0130 - accuracy: 0.9958 - val_loss: 0.4207 - val_accuracy: 0.9279
Epoch 63/100
281/281 [==============================] - 35s 123ms/step - loss: 0.0106 - accuracy: 0.9953 - val_loss: 0

[0.9450337886810303, 0.8830000162124634]

In [7]:
# Save the model
model_path = f'{MODEL_DIR}/{MODEL_NAME}_BS{BATCH_SIZE}_FT{FT_BLOCK*10}.h5'
model.save(model_path)

In [8]:
print("Model size in Mb:", getsize(model_path) / float(2**20))

Model size in Mb: 268.72528076171875
